Derived from sqlite_P1_v10

In [30]:
import pandas as pd
import sqlite3
import re as re
import mybib
from lxml import etree
from lxml import objectify
from sqlalchemy import create_engine
from subprocess import Popen

In [31]:
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 10)

In [32]:
p_mode='r' # r=replace, a = append

In [33]:
p_out_sql_L0='L0'
p_out_sql_L1='L1'
p_out_sql_L20='L20'
p_out_sql_resp='zzResp'

In [34]:
pfad_ddf=r'O:\Work\DV\MatthiasH\BBG_Tracking'.replace(chr(92),'/')
#file_ddf=pfad_ddf+'/'+'R315114383A_completes.ddf'
file_ddf=pfad_ddf+'/'+'data.ddf'

pfad_out=r'O:\Work\DV\MatthiasH\BBG_Tracking\out'.replace(chr(92),'/')
file_L0_out_csv=pfad_out+'/'+p_out_sql_L0+'.csv'
file_L1_out_csv=pfad_out+'/'+p_out_sql_L1+'.csv'
file_L20_out_csv=pfad_out+'/'+p_out_sql_L20+'.csv'
file_resp_out_csv=pfad_out+'/'+p_out_sql_resp+'.csv'


Get mdd info

In [35]:
path_mdd=r'\\TSHAMFIL901\Work\DV\MatthiasH\BBG_Tracking'.replace(chr(92),'/')
file_mdd=path_mdd+'/'+'R315114383A.mdd'

In [36]:
f = open(file_mdd,'r', encoding="utf8")
filedata = f.read()
f.close()
filecontent=filedata.replace('/Arc 3/2000-02-04','/Arc_3/2000_02_04')
filecontent=filecontent.replace('encoding="UTF-8"','')
tree = etree.fromstring(filecontent)

In [37]:
list_categoryids=tree.findall('.//categorymap/categoryid')

dfcat = pd.DataFrame()
for categoryid in list_categoryids:
    dfcat = dfcat.append(pd.Series([categoryid.attrib['value'],categoryid.attrib['name']]),ignore_index=True)
dfcat.columns=['CatValue','CatName']
dfcat.CatValue=dfcat.CatValue.astype(int)

#varcode=re.compile('^(_[0-9]{1,7})$')
#dfcat['Code']=dfcat.CatName.str.extract(varcode, expand=True)[0].str[1:].fillna(-1).astype(int)
#varcode=re.compile('(.*?)(_[0-9]{1,7})(.*?)')
#dfcat['Code2']=dfcat.CatName.str.extract(varcode, expand=True)[1].str[1:].fillna(-1).astype(int)

dict_mdd=dict(zip(dfcat['CatValue'].astype(int),dfcat['CatName']))

In [38]:
dict_mdd

{1: 'scan',
 2: 'htmlplayer',
 3: 'cati_htmlplayer',
 4: 'cati_ttyplayer',
 5: 'other',
 6: 'completed',
 7: 'active',
 8: 'timedout',
 9: 'scriptstopped',
 10: 'respondentstopped',
 11: 'shutdown',
 12: 'reviewed',
 13: 'signal',
 14: 'test',
 15: 'paper',
 16: 'sif',
 17: 'clean',
 18: 'needsreview',
 19: 'flaggedforreview',
 20: 'flaggedformove',
 21: 'flaggedforcopy',
 22: 'flaggedfordelete',
 23: 'dataentryplayer',
 24: 'initial',
 25: 'full',
 26: 'partial',
 27: 'maintenance',
 28: 'web',
 29: 'compclosed5',
 30: 'compclosed6',
 31: 'compclosed7',
 32: 'compclosed8',
 33: 'compclosed9',
 34: 'incomplete',
 35: 'incompatible',
 36: 'overquota',
 37: 'trapscreenout',
 38: 'tnsscreenout',
 39: 'ts_notengaged',
 40: 'enabled',
 41: 'disabled',
 42: 'pc',
 43: 'largetablet',
 44: 'largefeaturephone',
 45: 'mediumtablet',
 46: 'smalltablet',
 47: 'smallfeaturephone',
 48: 'smartphonetouch',
 49: 'unknown',
 50: 'yes',
 51: 'no',
 52: 'm',
 53: 'f',
 54: 'ipl01',
 55: 'ssi02',
 56: 'er

Get data

In [39]:
engine_dv_bbg = create_engine('mssql+pyodbc://TSMMHSQVS901/DV_BBG?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes')
p_if_exists='fail'
if p_mode=='r':
    p_if_exists='replace'
if p_mode=='a':
    p_if_exists='append'

In [40]:
path_bat=r'O:\Work\DV\MatthiasH\BBG_Tracking\out'+chr(92)
file_bat=path_bat+'/'+'bbg.bat'

In [55]:
# Read AnswerLabels from Database:
dflabel = pd.read_sql_query("""SELECT [DimVar] as Variable,[DimVal] as Code,[Label] FROM dbo.AnswerLabel""", engine_dv_bbg)

In [56]:
dflabel

Variable               Code                              Label
0          DimVar             DimVal                              Label
1    zzFlagSample                 _1                                LSR
2    zzFlagSample                 _2                                GMI
3    zzFlagSample                 _3                             Toluna
4          zzwave              wave1                            Welle 1
..            ...                ...                                ...
437        Status  RespondentStopped           Vom Antworter angehalten
438        Status           Shutdown  Befragungssystem heruntergefahren
439        Status           Reviewed                         Durchsicht
440        Status             Signal                             Signal
441        Status               Test                          Testdaten

[442 rows x 3 columns]

In [45]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(file_ddf)
df = pd.read_sql_query("SELECT * FROM L1", con)
con.close()

In [13]:
dfc1=df[df.columns[df.columns.str.contains(':C1$|^Respondent.Serial:L')]].copy()
dfs=df[df.columns[df.columns.str.contains(':S$|^Respondent.Serial:L')]].copy()
dfx=df[df.columns[df.columns.str.contains(':X$|^Respondent.Serial:L')]].copy()
#dfc1=df[df.columns[df.columns.str.contains(':C1$|^respid:|_wgt|^age_|^pim_')]].copy()
dict_c1=dict(zip(dfc1.columns,dfc1.columns.str.replace('(:S$|:C1$|:L$|:D$|\.Serial:L$)','')))
#dict_c1['brand_timeseries:C1']='BTS'
dict_c1['respid:L']='RespID'
dfc1.rename(columns=dict_c1, inplace=True)
dict_s=dict(zip(dfs.columns,dfs.columns.str.replace('(:S$|:C1$|:L$|:D$|\.Serial:L$)','')))
dfs.rename(columns=dict_s, inplace=True)

In [14]:
dfc1t=pd.melt(dfc1,id_vars=['Respondent'],var_name='Variable', value_name='Value')

In [15]:
dfst=pd.melt(dfs,id_vars=['Respondent'],var_name='Variable', value_name='Value')

In [16]:
dfst=dfst[dfst.Value>='0'].copy()

In [17]:
dfst2=pd.DataFrame(dfst.Value.str.split(';').tolist()).head()

In [18]:
dfst2=mybib.tidy_split(dfst, 'Value', sep=';')

In [19]:
dfst2=dfst2[dfst2.Value>'0'].copy()

In [20]:
dfL0=pd.concat([dfc1t,dfst2])


In [21]:
dfc1t['Code']=dfc1t.Value.astype(int).replace(dict_mdd)


#dfc1t

Variable      Q8a  S0   S2  S4  S6 S9a  ...   zzMainUser_h zzSchwarzbier  \
Respondent                              ...                                
15             _5  _1   _9  _1  _1  _3  ...             _1            _2   
20            _15  _1   _7  _3  _1  _3  ...             _1            _2   
22             _3  _1  _10  _2  _1  _3  ...             _1            _2   
23            _32  _1  _10  _1  _1  _3  ...             _1            _2   
24             _3  _1  _16  _2  _1  _3  ...             _2            _2   
...           ...  ..  ...  ..  ..  ..  ...            ...           ...   
7078          _27  _2   _7  _1  _1  _3  ...             _3            _2   
7116           _5  _2   _7  _3  _1  _3  ...             _1            _2   
7199        other  _2  _10  _3  _1  _3  ...             _1            _2   
7387           _2  _2  _10  _1  _2  _3  ...             _4            _2   
7468           _1  _2  _10  _3  _2  _3  ...             _1            _2   

Variable   zzgender zzregion zztotal zzwave  
Respondent                                   
15               _1       _9      _1  wave1  
20               _1       _7      _1  wave1  
22               _1      _10      _1  wave1  
23               _1      _10      _1  wave1  
24               _1      _16      _1  wave1  
...             ...      ...     ...    ...  
7078             _2       _7      _1  wave1  
7116             _2       _7      _1  wave1  
7199             _2      _10      _1  wave1  
7387             _2      _10      _1  wave1  
7468             _2      _10      _1  wave1  

[815 rows x 26 columns]

In [57]:
dfc1tl=dfc1t.merge(dflabel, how='left', on=('Variable','Code'), suffixes=('_x', '_y'))

In [58]:
dfc1tl

Respondent      Variable  Value Code               Label
0              15  zzFlagSample    182   _1                 LSR
1              20  zzFlagSample    182   _1                 LSR
2              22  zzFlagSample    182   _1                 LSR
3              23  zzFlagSample    182   _1                 LSR
4              24  zzFlagSample    182   _1                 LSR
...           ...           ...    ...  ...                 ...
21185        7078  zzMainUser_h    184   _3  Alkoholfreies Pils
21186        7116  zzMainUser_h    182   _1           Pils (AH)
21187        7199  zzMainUser_h    182   _1           Pils (AH)
21188        7387  zzMainUser_h    185   _4       Spezialitõten
21189        7468  zzMainUser_h    182   _1           Pils (AH)

[21190 rows x 5 columns]

In [59]:
dfresp=dfc1tl.pivot(index='Respondent', columns='Variable', values='Label').copy()
dfresp

Variable    Q8a        S0                 S2  \
Respondent                                     
15          NaN  Mõnnlich  Baden-W³rttemberg   
20          NaN  Mõnnlich    Rheinland-Pfalz   
22          NaN  Mõnnlich             Bayern   
23          NaN  Mõnnlich             Bayern   
24          NaN  Mõnnlich            Sachsen   
...         ...       ...                ...   
7078        NaN  Weiblich    Rheinland-Pfalz   
7116        NaN  Weiblich    Rheinland-Pfalz   
7199        NaN  Weiblich             Bayern   
7387        NaN  Weiblich             Bayern   
7468        NaN  Weiblich             Bayern   

Variable                                                   S4  \
Respondent                                                      
15          lõndlicher Gegend oder in einem Dorf (bis 5000...   
20          einer kleinen oder mittelgro¯en Stadt (20.000 ...   
22                einer Kleinstadt (5000 bis unter 20.000 EW)   
23          lõndlicher Gegend oder in einem Dorf (bis 5000...   
24                einer Kleinstadt (5000 bis unter 20.000 EW)   
...                                                       ...   
7078        lõndlicher Gegend oder in einem Dorf (bis 5000...   
7116        einer kleinen oder mittelgro¯en Stadt (20.000 ...   
7199        einer kleinen oder mittelgro¯en Stadt (20.000 ...   
7387        lõndlicher Gegend oder in einem Dorf (bis 5000...   
7468        einer kleinen oder mittelgro¯en Stadt (20.000 ...   

Variable                                          S6  S9a   ...     \
Respondent                                                  ...      
15                            _berwiegend ich selbst  NaN   ...      
20                            _berwiegend ich selbst  NaN   ...      
22                            _berwiegend ich selbst  NaN   ...      
23                            _berwiegend ich selbst  NaN   ...      
24                            _berwiegend ich selbst  NaN   ...      
...                                              ...  ...   ...      
7078                          _berwiegend ich selbst  NaN   ...      
7116                          _berwiegend ich selbst  NaN   ...      
7199                          _berwiegend ich selbst  NaN   ...      
7387        Ich und eine andere Person gleicherma¯en  NaN   ...      
7468        Ich und eine andere Person gleicherma¯en  NaN   ...      

Variable          zzMainUser_h zzSchwarzbier  zzgender           zzregion  \
Respondent                                                                  
15                   Pils (AH)            no  Mõnnlich  Baden-W³rttemberg   
20                   Pils (AH)            no  Mõnnlich    Rheinland-Pfalz   
22                   Pils (AH)            no  Mõnnlich             Bayern   
23                   Pils (AH)            no  Mõnnlich             Bayern   
24                 Weizen (AH)            no  Mõnnlich            Sachsen   
...                        ...           ...       ...                ...   
7078        Alkoholfreies Pils            no   Weiblch    Rheinland-Pfalz   
7116                 Pils (AH)            no   Weiblch    Rheinland-Pfalz   
7199                 Pils (AH)            no   Weiblch             Bayern   
7387             Spezialitõten            no   Weiblch             Bayern   
7468                 Pils (AH)            no   Weiblch             Bayern   

Variable   zztotal   zzwave  
Respondent                   
15           Total  Welle 1  
20           Total  Welle 1  
22           Total  Welle 1  
23           Total  Welle 1  
24           Total  Welle 1  
...            ...      ...  
7078         Total  Welle 1  
7116         Total  Welle 1  
7199         Total  Welle 1  
7387         Total  Welle 1  
7468         Total  Welle 1  

[815 rows x 26 columns]

In [60]:
dfresp.to_csv(file_resp_out_csv,sep='\t',index=True, header=False)
try:
    dfresp[:0].to_sql(p_out_sql_resp,engine_dv_bbg, if_exists=p_if_exists, index=True)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_resp], cwd=path_bat)
stdout, stderr = p.communicate()

In [22]:
dfL0.Value=dfL0.Value.astype(int)
dfL0['Code']=dfL0.Value.replace(dict_mdd)

In [23]:
dfL0.to_csv(file_L0_out_csv,sep='\t',index=False)
try:
    dfL0[:0].to_sql(p_out_sql_L0,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_L0], cwd=path_bat)
stdout, stderr = p.communicate()

In [24]:
con = sqlite3.connect(file_ddf)
levels = pd.read_sql_query("SELECT * FROM Levels ", con)
con.close()

In [25]:
Parent_L1_Tables=levels[(levels.ParentName=='L1')&(levels.DSCTableName.str[-5:]!='_bans')&(levels.DSCTableName.str[:5]!='Dummy')].DSCTableName.copy()
Parent_L1_Tables=Parent_L1_Tables.reset_index(drop=True)
Parent_L20_Tables=levels[(levels.ParentName=='L20')&(levels.DSCTableName.str[-5:]!='_bans')&(levels.DSCTableName.str[:5]!='Dummy')].DSCTableName.copy()
Parent_L20_Tables=Parent_L20_Tables.reset_index(drop=True)

In [26]:
dfL1=pd.DataFrame([])
dfTables=pd.DataFrame([])

for TableName in Parent_L1_Tables.values.tolist():
    #create dictionary from mdd-info to replace column names with their full qualified name
    #dict_VarL=dict(zip(mys[mys.Variable.str[:len(TableName)+5]==TableName+'[..].'].Variable.str[len(TableName)+5:],
             #mys[mys.Variable.str[:len(TableName)+5]==TableName+'[..].'].Variable))
    dict_levels=dict(zip(levels.DSCTableName,levels.TableName))
    TableName.replace(TableName,dict_levels[TableName])
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(file_ddf)
    Table=TableName.replace(TableName,dict_levels[TableName])
    df = pd.read_sql_query("SELECT [Respondent.Serial:L] as Respondent, "+Table+".* from "+Table+"  JOIN L1 ON L1.[:P0]="+Table+".[:P1] ", con)
    con.close()
 
    print ([Table]+df.columns.tolist())
    dfnewline=pd.DataFrame([Table]+df.columns.tolist()).T
    dfTables=pd.concat([dfTables,dfnewline])

    #Seperate :S (Multi) and :C1 (Single) Variables 
    dfs=df[df.columns[df.columns.str.contains(':S$|Respondent|LevelId')]].copy()
    dfc1=df[df.columns[df.columns.str.contains(':C1$|Respondent|:P[12]')]].copy()

    #Exclude banner variables
    if 'ban:S' in dfs.columns:
        dfs=dfs.drop('ban:S', axis=1)
    #Create dictionary to strip :C/:C1 Extensions
    dict_c1=dict(zip(dfc1.columns,dfc1.columns.str.replace('(:C1|\.Serial:L)','')))
    #Apply dictionaries to standardise column names according to mdd-info
    dfc1.rename(columns=dict_c1, inplace=True)
    #dfc1.rename(columns=dict_VarL, inplace=True)
    #Add column with TableName as constant value
    dfc1['Level']=TableName
    #Bring single punch data in long format:
    dfc1m=pd.melt(dfc1,id_vars=['Respondent','Level','LevelId',':P1'],var_name='Variable', value_name='Value')
    #Add column with TableName as constant value
    #dfc1m['Level']=TableName
    #Add column to indicate info type as 'Single'
    dfc1m['Type']='Single'

    dict_s=dict(zip(dfs.columns,dfs.columns.str.replace('(:C1|:S|\.Serial:L)','')))
    #Apply dictionaries to standardise column names according to mdd-info
    dfs.rename(columns=dict_s, inplace=True)
    #dfs.rename(columns=dict_VarL, inplace=True)
    dfs.loc[:,'Level']=TableName
    #Bring multi punch data in long format:
    dfsm=pd.melt(dfs,id_vars=['Respondent','Level','LevelId'],var_name='Variable', value_name='Value')
    dfsmt=pd.DataFrame([])
    if dfsm.Respondent.count()>0:
        dfsmt=dfsm.drop(['Value'], axis=1).join(dfsm['Value'].str.split(';',expand=True).stack().reset_index(drop=True, level=1).rename('Value') )
        dfsmt=dfsmt[dfsmt.Value>' ']
        dfsmt['Value']=dfsmt['Value'].astype(int)
        dfsmt['Respondent']=dfsmt['Respondent'].astype(int)
        dfsmt['LevelId']=dfsmt['LevelId'].astype(int)
        dfsmt['Type']='Multi'

    dfL1=pd.concat([dfL1,dfc1m])
    dfL1=pd.concat([dfL1,dfsmt])


['L2', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:C1']
['L5', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:X', 'slice.Coding:S']
['L8', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:X', 'slice.Coding:S']
['L9', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:S', 'slice.ans:L']
['L10', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:C1']
['L11', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:C1']
['L12', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'start:L', 'share:D', 'above:D', 'below:D', 'rank:D', 'shar_rank:D', 'final:D']
['L13', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:S']
['L14', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:S']
['L15', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:S']
['L16', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:S']
['L17', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'Q14a:X', 'Q14b_1:S']
['L18', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:C1']
['L19', 'Respondent', ':P1', ':P0', 'LevelId:C1', 'slice:C1']
['L20', 'Respo

In [27]:
dfTables

0           1    2    3           4         5               6    7    8   \
0    L2  Respondent  :P1  :P0  LevelId:C1  slice:C1             NaN  NaN  NaN   
0    L5  Respondent  :P1  :P0  LevelId:C1   slice:X  slice.Coding:S  NaN  NaN   
0    L8  Respondent  :P1  :P0  LevelId:C1   slice:X  slice.Coding:S  NaN  NaN   
0    L9  Respondent  :P1  :P0  LevelId:C1   slice:S     slice.ans:L  NaN  NaN   
0   L10  Respondent  :P1  :P0  LevelId:C1  slice:C1             NaN  NaN  NaN   
..  ...         ...  ...  ...         ...       ...             ...  ...  ...   
0   L26  Respondent  :P1  :P0  LevelId:C1   slice:S             NaN  NaN  NaN   
0   L27  Respondent  :P1  :P0  LevelId:C1   slice:S             NaN  NaN  NaN   
0   L28  Respondent  :P1  :P0  LevelId:C1   slice:S             NaN  NaN  NaN   
0   L29  Respondent  :P1  :P0  LevelId:C1   slice:S             NaN  NaN  NaN   
0   L30  Respondent  :P1  :P0  LevelId:C1   slice:S             NaN  NaN  NaN   

     9    10   11  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
..  ...  ...  ...  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  
0   NaN  NaN  NaN  

[23 rows x 12 columns]

In [28]:
dfs=df[df.columns[df.columns.str.contains(':S$|Respondent|LevelId')]].copy()

In [29]:
dfL1[dfL1.Type=='Multi']

:P1   Level  LevelId  Respondent   Type  Value      Variable
0     NaN      Q1      182          15  Multi  961.0  slice.Coding
20    NaN      Q1      182          20  Multi  863.0  slice.Coding
21    NaN      Q1      183          20  Multi  913.0  slice.Coding
22    NaN      Q1      184          20  Multi  951.0  slice.Coding
40    NaN      Q1      182          22  Multi  919.0  slice.Coding
...   ...     ...      ...         ...    ...    ...           ...
3255  NaN  MDN2_f      185        6622  Multi  190.0         slice
3256  NaN  MDN2_f      185        6623  Multi  190.0         slice
3257  NaN  MDN2_f      185        6684  Multi  190.0         slice
3258  NaN  MDN2_f      185        6758  Multi  190.0         slice
3259  NaN  MDN2_f      185        6762  Multi  190.0         slice

[191377 rows x 7 columns]

In [30]:
dfL1.Value=dfL1.Value.astype(int)
dfL1.LevelId=dfL1.LevelId.astype(int)
dfL1['Code']=dfL1.Value.replace(dict_mdd)
dfL1['LevelCode']=dfL1.LevelId.replace(dict_mdd)

In [31]:
dfL1.to_csv(file_L1_out_csv,sep='\t',index=False)
try:
    dfL1[:0].to_sql(p_out_sql_L1,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_L1], cwd=path_bat)
stdout, stderr = p.communicate()

In [32]:
dfL20=pd.DataFrame([])
dfTablesL20=pd.DataFrame([])


for TableName in Parent_L20_Tables.values.tolist():
    #create dictionary from mdd-info to replace column names with their full qualified name
    #dict_VarL=dict(zip(mys[mys.Variable.str[:len(TableName)+5]==TableName+'[..].'].Variable.str[len(TableName)+5:],
             #mys[mys.Variable.str[:len(TableName)+5]==TableName+'[..].'].Variable))
    dict_levels=dict(zip(levels.DSCTableName,levels.TableName))
    TableName.replace(TableName,dict_levels[TableName])
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(file_ddf)
    Table=TableName.replace(TableName,dict_levels[TableName])
    df = pd.read_sql_query("SELECT [Respondent.Serial:L] as Respondent, "+Table+".* from "+Table+"  JOIN L1 ON L1.[:P0]="+Table+".[:P1] ", con)
    con.close()

    print ([Table]+df.columns.tolist())
    dfnewline=pd.DataFrame([Table]+df.columns.tolist()).T
    dfTablesL20=pd.concat([dfTablesL20,dfnewline])

    
    #Seperate :S (Multi) and :C1 (Single) Variables 
    dfs=df[df.columns[df.columns.str.contains(':S$|Respondent|LevelId')]].copy()
    dfc1=df[df.columns[df.columns.str.contains(':C1$|Respondent')]].copy()

    #Exclude banner variables
    if 'ban:S' in dfs.columns:
        dfs=dfs.drop('ban:S', axis=1)
    #Create dictionary to strip :C/:C1 Extensions
    dict_c1=dict(zip(dfc1.columns,dfc1.columns.str.replace('(:C1|\.Serial:L)','')))
    #Apply dictionaries to standardise column names according to mdd-info
    dfc1.rename(columns=dict_c1, inplace=True)
    #dfc1.rename(columns=dict_VarL, inplace=True)
    #Add column with TableName as constant value
    dfc1['Level']=TableName
    #Bring single punch data in long format:
    dfc1m=pd.melt(dfc1,id_vars=['Respondent','Level','LevelId'],var_name='Variable', value_name='Value')
    #Add column with TableName as constant value
    #dfc1m['Level']=TableName
    #Add column to indicate info type as 'Single'
    dfc1m['Type']='Single'

    dict_s=dict(zip(dfs.columns,dfs.columns.str.replace('(:C1|:S|\.Serial:L)','')))
    #Apply dictionaries to standardise column names according to mdd-info
    dfs.rename(columns=dict_s, inplace=True)
    #dfs.rename(columns=dict_VarL, inplace=True)
    dfs.loc[:,'Level']=TableName
    #Bring multi punch data in long format:
    dfsm=pd.melt(dfs,id_vars=['Respondent','Level','LevelId'],var_name='Variable', value_name='Value')
    dfsmt=pd.DataFrame([])
    if dfsm.Respondent.count()>0:
        dfsmt=dfsm.drop(['Value'], axis=1).join(dfsm['Value'].str.split(';',expand=True).stack().reset_index(drop=True, level=1).rename('Value') )
        dfsmt=dfsmt[dfsmt.Value>' ']
        dfsmt['Value']=dfsmt['Value'].astype(int)
        dfsmt['Respondent']=dfsmt['Respondent'].astype(int)
        dfsmt['LevelId']=dfsmt['LevelId'].astype(int)
        dfsmt['Type']='Multi'

    dfL20=pd.concat([dfL20,dfc1m])
    dfL20=pd.concat([dfL20,dfsmt])


['L21', 'Respondent', ':P2', ':P1', ':P0', 'LevelId:C1', 'slice:C1']
['L22', 'Respondent', ':P2', ':P1', ':P0', 'LevelId:C1', 'slice:C1']


In [33]:
dfL20.Value=dfL20.Value.astype(int)
dfL20.LevelId=dfL20.LevelId.astype(int)
dfL20['Code']=dfL20.Value.replace(dict_mdd)
dfL20['LevelCode']=dfL20.LevelId.replace(dict_mdd)

In [34]:
dfL20.to_csv(file_L20_out_csv,sep='\t',index=False)
try:
    dfL20[:0].to_sql(p_out_sql_L20,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_L20], cwd=path_bat)
stdout, stderr = p.communicate()